In [ ]:
import pandas as pd
import numpy as np
import talib
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Load your dataset
df = pd.read_csv(r"C:\Users\devan\Downloads\BTC_2019_2023_1h.csv")

# Preprocess the dataset
def preprocess_data(df):
    df['datetime'] = pd.to_datetime(df['datetime'])
    
    # Calculate technical indicators
    df['RSI'] = talib.RSI(df['close'], timeperiod=14)
    df['EMA_20'] = talib.EMA(df['close'], timeperiod=20)
    df['EMA_50'] = talib.EMA(df['close'], timeperiod=50)
    df['ATR'] = talib.ATR(df['high'], df['low'], df['close'], timeperiod=14)
    df['MACD'], df['MACD_signal'], _ = talib.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)
    df['BB_upper'], df['BB_middle'], df['BB_lower'] = talib.BBANDS(df['close'], timeperiod=20)
    df['OBV'] = talib.OBV(df['close'], df['volume'])
    df['Stochastic_k'], df['Stochastic_d'] = talib.STOCH(df['high'], df['low'], df['close'])

    # Drop rows with NaN values (caused by indicators calculation)
    df = df.dropna()

    # Normalize the data
    scaler = MinMaxScaler()
    df[['close', 'volume', 'RSI', 'EMA_20', 'EMA_50', 'ATR', 'MACD', 'MACD_signal', 'OBV', 'Stochastic_k', 'Stochastic_d']] = scaler.fit_transform(
        df[['close', 'volume', 'RSI', 'EMA_20', 'EMA_50', 'ATR', 'MACD', 'MACD_signal', 'OBV', 'Stochastic_k', 'Stochastic_d']])
    
    return df

# Function to create sequences of a given window size
def create_sequences(data, window_size):
    sequences = []
    for i in range(len(data) - window_size):
        sequences.append(data[i:i + window_size])
    return np.array(sequences)

# LSTM Model for market regime classification
def create_lstm_model(input_shape):
    model = tf.keras.models.Sequential([
        tf.keras.layers.LSTM(128, return_sequences=False, input_shape=input_shape),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')  # Predict market regime between 0 and 1
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Assign market regime based on prediction
def assign_market_regime(prediction):
    if prediction <= 0.2:
        return "Strong Bear Market"
    elif 0.2 < prediction <= 0.4:
        return "Bear Market"
    elif 0.4 < prediction <= 0.6:
        return "Sideways Market"
    elif 0.6 < prediction <= 0.8:
        return "Weak Bull Market"
    else:
        return "Strong Bull Market"

# Generate trading signals based on market regime
def generate_trade_signals(df, regime):
    signals = []
    for i in range(1, len(df)):
        if regime == "Strong Bear Market":
            if df['RSI'].iloc[i] < 25 and df['MACD'].iloc[i] < df['MACD_signal'].iloc[i]:
                signals.append("Buy")
            else:
                signals.append("Hold")
        elif regime == "Bear Market":
            if df['RSI'].iloc[i] < 30 and df['MACD'].iloc[i] > df['MACD_signal'].iloc[i]:
                signals.append("Buy")
            else:
                signals.append("Hold")
        elif regime == "Sideways Market":
            if df['close'].iloc[i] < df['BB_lower'].iloc[i]:
                signals.append("Buy")
            elif df['close'].iloc[i] > df['BB_upper'].iloc[i]:
                signals.append("Sell")
            else:
                signals.append("Hold")
        elif regime == "Weak Bull Market":
            if df['RSI'].iloc[i] > 50 and df['MACD'].iloc[i] > df['MACD_signal'].iloc[i]:
                signals.append("Buy")
            else:
                signals.append("Hold")
        elif regime == "Strong Bull Market":
            if df['RSI'].iloc[i] > 60:
                signals.append("Buy")
            else:
                signals.append("Hold")
    return signals

# Simulate trading based on generated signals
def simulate_trading(signals, df):
    position = None
    trades = []
    for i, signal in enumerate(signals):
        if signal == "Buy" and position is None:
            position = df['close'].iloc[i]
            trades.append(('Buy', df['close'].iloc[i]))
        elif signal == "Sell" and position is not None:
            trades.append(('Sell', df['close'].iloc[i]))
            position = None
    return trades

# Main execution flow
df = preprocess_data(df)

# Create sequences for LSTM training
window_size = 60  # Look-back window size (e.g., 60 timesteps)
X = create_sequences(df[['close', 'volume', 'RSI', 'EMA_20', 'EMA_50', 'ATR', 'MACD', 'MACD_signal', 'OBV', 'Stochastic_k', 'Stochastic_d']].values, window_size)
y = np.random.uniform(0, 1, size=(len(X)))  # Use real market regime labels for training

# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Build and train the LSTM model
model = create_lstm_model((window_size, X.shape[2]))
model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

# Predict market regimes
market_regime_predictions = model.predict(X_test)
market_regime = [assign_market_regime(pred) for pred in market_regime_predictions]

# Generate trade signals based on the predicted regimes
df['Market Regime'] = market_regime
trade_signals = generate_trade_signals(df, df['Market Regime'].iloc[0])

# Simulate trading based on generated signals
trades = simulate_trading(trade_signals, df)

# Example of trade output
print(trades)
